<a href="https://colab.research.google.com/github/kim-yujin497/kim-yujin497.github.io/blob/main/ML%ED%81%B4%EB%9E%98%EC%8A%A4%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_FBI_fire_decetion_by_yolov3%EC%9D%98%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 사용 설명
darkent의 yolo v3를 이용한 화재 검출 모델입니다.  
C,CUDA로 빌드된 모듈을 그대로 사용했으며  
Colab에서의 사용을 권장드립니다
  

  
https://github.com/AlexeyAB/darknet 를 이용했습니다.


## Darknet 빌드

In [ ]:
!git clone https://github.com/AlexeyAB/darknet 

In [ ]:
%cd darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile
# 동영상 테스트 할때만 OPENCV 를 1로 설정해주세요
# !sed -i 's/OPENCV=0/OPENCV=1/g' Makefile


!cat Makefile
!make

## Dataset 불러오기

화재사진, 화재 동영상들을 크롤링 한 뒤  
직접 BoundBox를 라벨링 해 주었습니다.  

https://drive.google.com/openid=1bT6jGRyNnqoaQTD5v4mcEDshRB2NkBQY

위의 드라이브를 개인 드라이브에 직접 복사해주세요 ! 

In [ ]:
import os.path
import shutil
from google.colab import drive

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive',force_remount=True)
  
DOWNLOAD_LOCATION = '/content/darknet/data/'
DRIVE_DATASET_FILE = '/content/gdrive/My Drive/FBI_dataset.zip

shutil.copy(DRIVE_DATASET_FILE, DOWNLOAD_LOCATION)


print('Successfully downloaded the dataset')

### Unzip the dataset

In [ ]:
!unzip /content/darknet/data/FBI_dataset.zip -d data/ 

## Yolo v3 가중치 가져오기


In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74  

## cfg 파일 수정
### 아래는 Darknet README에 있는 권장사항입니다.


- batch
- subdivisions (if you get memory out error, increase this 16, 32 or 64)
- max_batches (it should be classes*2000)
- steps (it should be 80%, 90% of max_batches)
- classes (the number of classes which you are going to train)
- filters (the value for filters can be calculated using (classes + 5)x3 )


In [ ]:
%cd /content/darknet
!sed -i 's/batch=1/batch=64/g' cfg/yolov3.cfg
!sed -i 's/subdivisions=1/subdivisions=16/g' cfg/yolov3.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/g' cfg/yolov3.cfg
!sed -i 's/steps=400000,450000/steps=4800,5400/g' cfg/yolov3.cfg
!sed -i 's/classes=80/classes=3/g' cfg/yolov3.cfg
!sed -i 's/filters=255/filters=24/g' cfg/yolov3.cfg
!sed -i 's/width=416/width=224/g' cfg/yolov3.cfg
!sed -i 's/height=416/height=224/g' cfg/yolov3.cfg



!cat cfg/yolov3.cfg

## 학습되었던 가중치 가져오기  

- 이전에 train 한 가중치가 있다면 아래 cell을 실행해 주세요




In [ ]:
# 이전에 train 한 가중치가 있을 때 아래 cell을 실행해 주세요



import os.path
import shutil
from google.colab import drive

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')
  
BACKUP_FOLDER = '/content/darknet/backup'
DRIVE_YOLO_BACKUP = '/content/gdrive/My Drive/ai/weight/yolov3_last.weights'

shutil.copy(DRIVE_YOLO_BACKUP, BACKUP_FOLDER)

print('Successfully fetched the pretrained files for Yolo from Google drive')

# 학습
1) 100 iter마다 yolo-obj_llast.weight가 저장됩니다  
2) 1000단위마다 yolo-obj_xxxx.weight가 저장됩니다

### 아래는 github 원문입니다
- file yolo-obj_last.weights will be saved to the darknet/backup for each 100 iterations
- file yolo-obj_xxxx.weights will be saved to the darknet/backup for each 1000 iterations
- After each 100 iterations if you want, you can stop and later start training from this point. For example, after 2000 iterations you can stop training, and later just start training using: darknet detector train data/obj.data yolov3.cfg backup/yolo-obj_2000.weights

### Train,Test 분리

- Model에 넣을 파일 명을 가져옵니다
- ( PNG확장자만 가능합니다 !)


In [ ]:
### train과 test 데이터 파일 리스트를 만들어주는 함수입니다


import os
import sys

CURRENT_DIR = '/content/darknet/data/img'

def progressBar(value, endvalue, bar_length=20):
  percent = float(value) / endvalue
  arrow = '-' * int(round(percent * bar_length)-1) + '>'
  spaces = ' ' * (bar_length - len(arrow))

  sys.stdout.write("\rPercent: [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
  sys.stdout.flush()

def returnFileList(dirname, extract):
  fileList = []
  filenames = os.listdir(dirname)
  for filename in filenames:
    ext = os.path.splitext(filename)[-1]
    if ext == extract: 
      fileList.append(filename)
  return fileList

fileNames = returnFileList(CURRENT_DIR, ".png")

from sklearn.model_selection import train_test_split
train,test = train_test_split(fileNames, test_size=0.1, random_state=123)
train,valid = train_test_split(train,test_size=0.1,random_state=123)


with open("/content/darknet/data/train.txt","w") as fileList:
  for i, names in enumerate(train):
    fileList.write(CURRENT_DIR+"/"+names+"\n")
    progressBar(i, len(train), bar_length=100)

with open("/content/darknet/data/valid.txt","w") as fileList:
  for i, names in enumerate(valid):
    fileList.write(CURRENT_DIR+"/"+names+"\n")
    progressBar(i, len(valid), bar_length=100)

with open("/content/darknet/data/test.txt","w") as fileList:
  for i, names in enumerate(test):
    fileList.write(CURRENT_DIR+"/"+names+"\n")
    progressBar(i, len(test), bar_length=100)

In [ ]:
# 처음부터 시작할 때, 이 코드를 실행해 주세요
!./darknet detector train data/obj.data cfg/yolov3.cfg darknet53.conv.74

# 저장된 weight가 있을 때, 이 코드를 실행해 주세요
#!./darknet detector train data/obj.data cfg/yolov3.cfg backup/yolov3_last.weights

### 학습이 끝난 마지막 weight 업로드 하기
- 경로는 각자의 환경에 맞게 수정해 주세요 !


In [ ]:
import os.path
import shutil
from google.colab import drive

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')
  
YOLO_BACKUP = '/content/darknet/backup/yolov3_last.weights' 
DRIVE_DIR = '/content/gdrive/My Drive/ai/weight' 

shutil.copy(YOLO_BACKUP, DRIVE_DIR)

print('Saved training data to drive at: ' + DRIVE_DIR)

# 평가하기

In [ ]:
#cd /content/darknet

In [ ]:
!./darknet detector map  data/obj.data cfg/yolov3.cfg backup/yolov3_last.weights

# Prediction 하기

In [ ]:
!./darknet detector test data/obj.data cfg/yolov3.cfg backup/yolov3_last.weights 'data/테스트이미지 이름.png' 

### 결과 출력

In [ ]:
def display_image(file_path = '/content/darknet/predictions.jpg'):
    import cv2
    import matplotlib.pyplot as plt
    import os.path

    if os.path.exists(file_path):
      img = cv2.imread(file_path)
      show_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
      plt.imshow(show_img)
    else:
      print('failed to open file')
    
display_image()